**Subset Sum Problem: Brute Force Approach**

The Subset Sum problem asks if a subset of a given set of numbers sums up to a specific target value. The brute-force approach, or exhaustive search, solves this by examining every single possible subset to check if its sum matches the target.

**Time Complexity**

the time complexity for the brute-force approach to the Subset Sum problem is $O(2^n)$; Exponential complexity, making it highly inefficient for large sets.

In [4]:
import itertools

In [5]:
def find_subset_sum_bruteforce(s: list, m: int) -> list:
    """
    Finds all subsets of a given set 's' that sum up to the target 'm'
    using the brute-force (exhaustive search) approach.
    Args:
        s (list): The list of numbers to search within.
        m (int): The target sum.

    Returns:
        list: A list of subsets (represented as lists) that equal the target sum.
    """

    n = len(s)
    solutions = []

    # Iterate from 0 up to (but not including) 2^n
    for i in range(1 << n):
        current_subset = []
        current_sum = 0
        
        # Check the j-th bit of 'i' to determine if the j-th element of 's'
        # is included (1) or excluded (0). This simulates the Solution Vector X.
        for j in range(n):
            # (i >> j) & 1 checks if the j-th bit is set
            if (i >> j) & 1:
                # Bit is 1 (element included, X[j]=1)
                current_subset.append(s[j])
                current_sum += s[j]

        # Check if the current subset is a solution
        if current_sum == m:
            solutions.append(current_subset)
            
    return solutions

In [7]:
#Examples: 
S = [1, 2, 3, 4]
M = 6
print(f"Set S: {S}")
print(f"Target Sum M: {M}")

# Find and display the solutions
result = find_subset_sum_bruteforce(S, M)

if result:
    print("\nFound Solutions (Subsets that sum to 6):")
    for solution in result:
        print(f"  {solution} (Sum: {sum(solution)})")
else:
    print("\nNo subset found that sums to the target.")

# Example 2: No solution
S2 = [10, 20, 30]
M2 = 5
result2 = find_subset_sum_bruteforce(S2, M2)
print(f"\nSet S: {S2}, Target Sum M: {M2}")
print(f"Result: {result2 if result2 else 'No solution found'}")

Set S: [1, 2, 3, 4]
Target Sum M: 6

Found Solutions (Subsets that sum to 6):
  [1, 2, 3] (Sum: 6)
  [2, 4] (Sum: 6)

Set S: [10, 20, 30], Target Sum M: 5
Result: No solution found


-------------------------------------------------------------------------------------------------------------------------------------------------------

**Subset Sum Problem: Dynamic Programming Approach**

The Dynamic Programming approach solves the Subset Sum problem by breaking it down into smaller, overlapping subproblems. We construct a table to store the results of these subproblems, preventing redundant calculations.

The solution uses a 2D Boolean table, $DP[i][j]$, where:
* Rows ($i$): Correspond to the elements of the set being considered, plus one row for the base case (the empty set).If the set $S$ has $n$ elements, the table has $n+1$ rows.
* Columns ($j$): Correspond to all possible target sums from $0$ up to the main target sum $M$.If the target sum is $M$, the table has $M+1$ columns (from index 0 to $M$).
* Value: $DP[i][j]$ is True if a subset of the first $i$ elements of $S$ sums up to $j$; otherwise, it is False.

In [11]:
def subset_sum_dp(L: list, T: int) -> tuple[bool, list]:
    """
    Solves the Subset Sum problem using the Dynamic Programming approach.

    Args:
        L (list): The set of numbers (e.g., [3, 5, 9, 4, 8]).
        T (int): The target sum (e.g., 16).

    Returns:
        tuple[bool, list]: A tuple where the first element is True if a subset
                           is found, and the second is the list of elements
                           forming that subset (or an empty list if not found).
    """
    n = len(L)
    
    # Initialize the DP table: (n+1) rows and (T+1) columns, all set to False
    # DP[i][j] will be True if a subset of L[:i] sums to j.
    dp = [[False] * (T + 1) for _ in range(n + 1)]

    # 1. Base Case: Empty set sums to 0
    # The leftmost column (j=0) is always True
    for i in range(n + 1):
        dp[i][0] = True

    # 2. Fill the rest of the table
    # Start from row 1 (first element) and column 1 (sum 1)
    for i in range(1, n + 1):
        current_element = L[i-1] # The i-th element (0-indexed)

        for j in range(1, T + 1):
            
            #  Case 1: Excluding the current element 
            # If the sum 'j' was possible without the current element, it's still possible.
            dp[i][j] = dp[i-1][j]

            # Case 2: Including the current element  
            # Check if the current element is smaller than or equal to the target sum 'j'
            if j >= current_element:
                # If the sum (j - current_element) was achievable in the previous row (i-1),
                # then sum 'j' is achievable by including the current element.
                # We use logical OR to combine the two possibilities (exclude OR include).
                remaining_sum_possible = dp[i-1][j - current_element]
                dp[i][j] = dp[i][j] or remaining_sum_possible
                
    # 3. Check the Final Result
    # The solution is in the bottom-right cell: dp[n][T]
    if not dp[n][T]:
        return False, []

    # 4. Backtracking to Find the Subset Elements
    # Start at the target cell (last row, last column)
    subset_elements = []
    i, j = n, T

    while i > 0 and j > 0:
        # Check if the sum 'j' was achieved *without* the current element (L[i-1])
        
        if dp[i-1][j]:
            # The current element L[i-1] was NOT needed. Move up a row.
            i -= 1
        else:
            # The current element L[i-1] *WAS* needed to achieve sum 'j'.
            # This follows the logic: subtract the row value (L[i-1]) to get the new column (j).
            # The current element is part of the solution.
            subset_elements.append(L[i-1])
            j -= L[i-1] # Move left by the element's value
            i -= 1     # Move up a row

    return True, subset_elements

In [12]:
#EExample
L = [3, 5, 9, 4, 8]
T = 16

found, subset = subset_sum_dp(L, T)

print(f"Set: {L}")
print(f"Target Sum: {T}")
print("---")

if found:
    print(f"✅ Subset found that sums to {T}.")
    print(f"Elements: {subset} (Check: {subset[0]} + {subset[1]} + {subset[2]} = {sum(subset)})")
else:
    print(f"❌ No subset found that sums to {T}.")

Set: [3, 5, 9, 4, 8]
Target Sum: 16
---
✅ Subset found that sums to 16.
Elements: [4, 9, 3] (Check: 4 + 9 + 3 = 16)


**Time Complexity**

The algorithm builds and traverses an $O(n \times T)$ table, where $n$ is the number of elements and $T$ is the target sum. Since each cell takes $O(1)$ time to fill, the overall time complexity is $\mathbf{O(n \cdot T)}$.